# Extracting Metadata from Chapters with Outlines 

In [19]:
import PyPDF2 as p2
import re
import numpy as np
import pandas as pd
from pprint import pprint

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preparing the Data 

In [12]:
df_outlines = pd.read_csv("F:\Environmental Baseline Data\Version 3\Indices\Outline_Present_htmlTOC.csv")
 
len(df_outlines)
df_outlines[20:30].head(5)

1328

,Unnamed: 0,file_nos,file_names,file_paths,int_postals,TOC_present,consecutive_postals,chunk_nos,outline_present
20,20,1,A0Y1V1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,62871,0,1,4,1
21,21,1,A0Y1V2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,62872,0,1,4,1
22,22,1,A0Y1V3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,62873,1,1,5,0
23,23,1,A0Y1V4.pdf,F:/Environmental Baseline Data/Version 3/Data/...,62874,1,1,6,0
24,24,1,A0Y1V5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,62875,1,1,7,1


## Get Chapter Names for all the PDFs with outlines

In [51]:
# Functtion that runs recursively to extract the outlines of the PDF files
def nested_is_instance_check(s, level, file_name, path, chunk_no,
                             PDF_names, PDF_locations, Chunk_Nos, Chapter_Numbers, 
                             Chapter_Keys, Chapter_levels, PDF_Found_Last, Page_Found_Last):
    level +=1
    if isinstance(s, (int, list, float, complex)):
        for x in s:
            nested_is_instance_check(x, level, file_name, path, chunk_no,
                                     PDF_names, PDF_locations, Chunk_Nos, Chapter_Numbers, 
                                     Chapter_Keys, Chapter_levels, PDF_Found_Last, Page_Found_Last)
    elif level < 5:
        PDF_names.extend([file_name])
        full_path =  path + file_name
        PDF_locations.extend([full_path])
        Chunk_Nos.extend([chunk_no])
        
        chapter_title = s.title
        Chapter_Numbers.extend([0])
        Chapter_Keys.extend([chapter_title])
        
        Chapter_levels.extend([level])
        #it was observed that for a lot of PDF files we could not find the page number 
        # Hence, -999 is the error code added for those cases 
        page_error_flag = 0
        try:
            Page_Found_Last.extend([(pdfread.getDestinationPageNumber(s))])
        except: 
            Page_Found_Last.extend([-999])
            page_error_flag = 1
        
        
        if page_error_flag != 1:
            PDF_Found_Last.extend([file_name])
        else:
            PDF_Found_Last.extend([' '])
            
        

In [52]:
PDF_names = []
PDF_locations = []
Chunk_Nos = []
Chapter_Numbers = []
Chapter_Keys = []
Chapter_levels = []
PDF_Found_Last = [] # This is redundant for Outline method, unless used with the chuck method 
Page_Found_Last = []
path = test2 = "F:/Environmental Baseline Data/Version 3/Data/PDF/"


for i in range(len(df_outlines)):
#     First file in the chunk has Outline 
#     print("PDF name {}".format(df_outlines.iloc[i]['file_names']))
    level = 0
    if df_outlines.iloc[i]['outline_present'] == 1:
        pdfread = p2.PdfFileReader(df_outlines.iloc[i]['file_paths'])
        s = pdfread.outlines
        nested_is_instance_check(s, 
                                level, 
                                df_outlines.iloc[i]['file_names'], path,
                                df_outlines.iloc[i]['chunk_nos'], 
                                PDF_names, PDF_locations, Chunk_Nos, Chapter_Numbers, 
                                Chapter_Keys, Chapter_levels, PDF_Found_Last, Page_Found_Last)
        #file_name_containing_index = current_df_outlines.iloc[i]['file_names']
            
    

PDF name A0H8C0.pdf
PDF name A0H8C1.pdf
PDF name A0H8C2.pdf
PDF name A0H8C3.pdf
PDF name A0U3G1.pdf
PDF name A0U3G2.pdf
PDF name A0Y1T7.pdf
PDF name A0Y1T8.pdf
PDF name A0Y1T9.pdf
PDF name A0Y1U0.pdf
PDF name A0Y1U1.pdf
PDF name A0Y1U2.pdf
PDF name A0Y1U3.pdf
PDF name A0Y1U4.pdf
PDF name A0Y1U5.pdf
PDF name A0Y1U6.pdf
PDF name A0Y1U7.pdf
PDF name A0Y1U8.pdf
PDF name A0Y1U9.pdf
PDF name A0Y1V0.pdf
PDF name A0Y1V1.pdf
PDF name A0Y1V2.pdf
PDF name A0Y1V3.pdf
PDF name A0Y1V4.pdf
PDF name A0Y1V5.pdf
PDF name A0Y1V6.pdf
PDF name A0Y1V7.pdf
PDF name A0Y1V8.pdf
PDF name A0Y1V9.pdf
PDF name A0Y1W0.pdf
PDF name A0Y1W1.pdf
PDF name A0Z0Y6.pdf
PDF name A0Z0Y7.pdf
PDF name A0Z0Y8.pdf
PDF name A0Z0Y9.pdf
PDF name A0Z0Z0.pdf
PDF name A0Z0Z1.pdf
PDF name A0Z0Z2.pdf
PDF name A0Z0Z3.pdf
PDF name A0Z0Z4.pdf
PDF name A0Z0Z5.pdf
PDF name A0Z0Z6.pdf
PDF name A0Z0Z7.pdf
PDF name A0Z0Z8.pdf
PDF name A0Z0Z9.pdf
PDF name A0Z1A0.pdf
PDF name A0Z1A1.pdf
PDF name A0Z1A2.pdf
PDF name A0Z1A3.pdf
PDF name A0Z1A4.pdf


PDF name A3Q6H8.pdf
PDF name A3Q6H9.pdf
PDF name A3Q6I0.pdf
PDF name A3Q6I1.pdf
PDF name A3Q6I2.pdf
PDF name A3Q6I3.pdf
PDF name A3Q6I4.pdf
PDF name A3Q6I5.pdf
PDF name A3Q6I6.pdf
PDF name A3Q6I7.pdf
PDF name A3Q6I8.pdf
PDF name A3Q6I9.pdf
PDF name A3Q6J0.pdf
PDF name A3Q6J1.pdf
PDF name A3Q6J2.pdf
PDF name A3Q6J3.pdf
PDF name A3Q6J4.pdf
PDF name A3Q6J5.pdf
PDF name A3Q6J6.pdf
PDF name A3Q6J7.pdf
PDF name A3Q6J8.pdf
PDF name A3Q6J9.pdf
PDF name A3Q6K0.pdf
PDF name A3Q6K1.pdf
PDF name A3Q6K2.pdf
PDF name A3Q6K3.pdf
PDF name A3Q6K4.pdf
PDF name A3Q6K5.pdf
PDF name A3Q6K6.pdf
PDF name A3S1T2.pdf
PDF name A3S1T3.pdf
PDF name A3S1T4.pdf
PDF name A3S1T5.pdf
PDF name A3S1T6.pdf
PDF name A3S1T7.pdf
PDF name A3S1T8.pdf
PDF name A3S1T9.pdf
PDF name A3S1U0.pdf
PDF name A3S1U1.pdf
PDF name A3S1U2.pdf
PDF name A3S1U3.pdf
PDF name A3S1U4.pdf
PDF name A3S1U5.pdf
PDF name A3S1U6.pdf
PDF name A3S1U7.pdf
PDF name A3S1U8.pdf
PDF name A3S1U9.pdf
PDF name A3S1V0.pdf
PDF name A3S1V1.pdf
PDF name A3S1V2.pdf


PDF name A5A7D9.pdf
PDF name A5A7E0.pdf
PDF name A5A7E1.pdf
PDF name A5A7E2.pdf
PDF name A5A7E7.pdf
PDF name A5A7E8.pdf
PDF name A5A7E9.pdf
PDF name A5A7F0.pdf
PDF name A5A7F1.pdf
PDF name A5A7F2.pdf
PDF name A5A7F3.pdf
PDF name A5A7F4.pdf
PDF name A5A7F5.pdf
PDF name A5A7F9.pdf
PDF name A5A7G0.pdf
PDF name A5A7G1.pdf
PDF name A5A7G2.pdf
PDF name A5A7G3.pdf
PDF name A5A7G4.pdf
PDF name A5A7G5.pdf
PDF name A5A7G6.pdf
PDF name A5A7G7.pdf
PDF name A5A7G8.pdf
PDF name A5A7H3.pdf
PDF name A5A7H4.pdf
PDF name A5A7H5.pdf
PDF name A5A7H6.pdf
PDF name A5A7H7.pdf
PDF name A5A7H8.pdf
PDF name A5A7H9.pdf
PDF name A5A7I0.pdf
PDF name A5A7I1.pdf
PDF name A5A7I2.pdf
PDF name A5A7I6.pdf
PDF name A5A7I7.pdf
PDF name A5A7I8.pdf
PDF name A5A7I9.pdf
PDF name A5A7J0.pdf
PDF name A5A7J1.pdf
PDF name A5A7J2.pdf
PDF name A5A7J3.pdf
PDF name A5A7J4.pdf
PDF name A5A7J5.pdf
PDF name A5A7J9.pdf
PDF name A5A7K0.pdf
PDF name A5A7K1.pdf
PDF name A5A7K2.pdf
PDF name A5A7K3.pdf
PDF name A5A7K4.pdf
PDF name A5A7K5.pdf


PDF name A5G2X0.pdf
PDF name A5G2X1.pdf
PDF name A5G2X2.pdf
PDF name A5G2X3.pdf
PDF name A5G2X4.pdf
PDF name A5G2X5.pdf
PDF name A5G2X6.pdf
PDF name A5G2X7.pdf
PDF name A5G2X8.pdf
PDF name A5G2X9.pdf
PDF name A5G2Y0.pdf
PDF name A5G2Y1.pdf
PDF name A5G2Y3.pdf
PDF name A5V8D5.pdf
PDF name A5V8D6.pdf
PDF name A5V8D7.pdf
PDF name A5V8D8.pdf
PDF name A5V8D9.pdf
PDF name A5V8E0.pdf
PDF name A5V8E1.pdf
PDF name A5V8E2.pdf
PDF name A5V8E3.pdf
PDF name A5V8E4.pdf
PDF name A5V8E5.pdf
PDF name A5V8E6.pdf
PDF name A5V8E7.pdf
PDF name A5V8E8.pdf
PDF name A5V8E9.pdf
PDF name A5V8F0.pdf
PDF name A5V8F1.pdf
PDF name A5V8F2.pdf
PDF name A5V8F3.pdf
PDF name A5V8F4.pdf
PDF name A5V8F5.pdf
PDF name A5V8F6.pdf
PDF name A5V8F7.pdf
PDF name A5V8F8.pdf
PDF name A5V8F9.pdf
PDF name A5V8G0.pdf
PDF name A5V8G1.pdf
PDF name A5V8G2.pdf
PDF name A6A2C8.pdf
PDF name A6A2C9.pdf
PDF name A6A2D0.pdf
PDF name A6A2D1.pdf
PDF name A6A2D2.pdf
PDF name A6A2D3.pdf
PDF name A6A2D4.pdf
PDF name A6A2D5.pdf
PDF name A6A2D6.pdf


In [53]:
len(PDF_names)
len(PDF_locations)
len(Chunk_Nos)
len(Chapter_Numbers)
len(Chapter_Keys)
len(Chapter_levels)
len(PDF_Found_Last)
len(Page_Found_Last)
len(Chapter_Keys)

12835

12835

12835

12835

12835

12835

12835

12835

12835

In [87]:
allChaptersDF = pd.DataFrame({'PDF_names' : PDF_names, 
                              'PDF_locations' : PDF_locations,
                              'Chunk_Nos' : Chunk_Nos,
                              'Chapter_Numbers' : Chapter_Numbers,  
                              'Chapter_Keys' : Chapter_Keys,
                              'Chapter_levels' : Chapter_levels,
                              'PDF_Found_Last': PDF_Found_Last,
                              'Page_Found_Last' : Page_Found_Last})
allChaptersDF.head(5)

,PDF_names,PDF_locations,Chunk_Nos,Chapter_Numbers,Chapter_Keys,Chapter_levels,PDF_Found_Last,Page_Found_Last
0,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Environmental Impact Assessment and Mitigation...,2,A0H8C0.pdf,24
1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Introduction,3,A0H8C0.pdf,24
2,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Project Background,4,A0H8C0.pdf,24
3,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Regulatory Setting,4,A0H8C0.pdf,24
4,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Scope of Environmental Assessment,4,A0H8C0.pdf,26


In [185]:
PositiveChaptersDF = allChaptersDF[allChaptersDF['Page_Found_Last'] > -1]
PositiveChaptersDF = allChaptersDF[allChaptersDF['Chapter_levels'] < 4 ]
# len(PositiveChaptersDF)
# PositiveChaptersDF.head(5)

PositiveChaptersDF = PositiveChaptersDF[allChaptersDF.Chunk_Nos.isin([274, 275])]
PositiveChaptersDF

,PDF_names,PDF_locations,Chunk_Nos,Chapter_Numbers,Chapter_Keys,Chapter_levels,PDF_Found_Last,Page_Found_Last
11259,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,0,Cover,2,A6F4Q3.pdf,0
11260,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,0,Master Table of Contents,3,A6F4Q3.pdf,2
11261,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,0,Acronyms and Abbreviations,3,A6F4Q3.pdf,4
11262,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,0,1.0 INTRODUCTION,3,A6F4Q3.pdf,13
11264,A6F4Q3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,274,0,2.0 PROJECT DESCRIPTION,3,A6F4Q3.pdf,27
...,...,...,...,...,...,...,...,...
11658,A6F4R2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,275,0,Appendix J: Preliminary Decommissioning and Ab...,2,A6F4R2.pdf,406
11659,A6F4R2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,275,0,Preliminary Decommissioning and Abandonment Plan,3,A6F4R2.pdf,407
11661,A6F4R2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,275,0,Appendix K: Traditional Knowledge Report,2,A6F4R2.pdf,429
11662,A6F4R2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,275,0,TABLE OF CONTENTS,3,A6F4R2.pdf,432


# 1. Extracting Text from the Chapter 

In [186]:
def remove_digits(chapter_key):
    """ A lot of thimes the chapter name in the index have a number at the start of the string, 
    however, when we are searching for that text in the PDF file the number and the text are seperated by /n
    Example: 
    Chapter Key in Outline : " 5.0 Environment Assessment  ""
    Chapter Key in actual PDF: "5.0
                            Environment Assessment"
    
    
    
    This function removes the digits if the and strips the text to improve the searcheability.
    New Chapter Key in Outline : "Environment Assessment"  
    """
    index = 0
    chapter_key = chapter_key.strip()
    if ' ' in chapter_key and chapter_key[0].isdigit():
        index = chapter_key.index(' ')
        chapter_key = chapter_key[index+1:]
    return(chapter_key.strip())
        
    

In [187]:
def read_chapter_text(PDF_location, Start_Text, Start_Page, End_Page, End_Text):
    
    print("Reading PDF {}".format(PDF_location))
    object = p2.PdfFileReader(PDF_location)
    
    Start_Text = remove_digits(Start_Text)
    End_Text = remove_digits(End_Text)
        
    # get number of pages
    NumPages = object.getNumPages()
    if NumPages < End_Page:
        End_Page = NumPages - 1
        
    
    
    Chapter = ""    
    for i in range(Start_Page, End_Page+1):
        PageObj = object.getPage(i)
        
#         print("*****************************************")
#         print("*****************************************")
#         print("*****************************************")
        
        try:
            Text = PageObj.extractText()
            Text = Text.replace('\n','').replace('\n','')
            #print("The Text Extracted for page no {} is {}".format(i, Text))
        except:
            Text = "Text could not be extracted for File {} for Page Number {}".format(PDF_location, i)
            print(Text)
            continue
        
        if Start_Page == End_Page+1:
            try:
                Text = Text.split(Start_Text)[1]
            except:
                 print("COULD NOT FIND {} IN THE PAGE WITH THE TEXT".format(Start_Text))
            try:
                Text = Text.split(End_Text)[0]
            except:#                 
                print("COULD NOT FIND {} IN THE PAGE WITH THE TEXT".format(End_Text))       

            Chapter = Start_Text + Text
            
        
        elif i == Start_Page:
            try:
                Text = Text.split(Start_Text)[1]
            except:
                print("Never Mind: COULD NOT FIND {} IN THE PAGE WITH THE TEXT".format(Start_Text))
            Chapter = Start_Text + Text
            
        elif i == End_Page + 1:
            try:
                Text = Text.split(End_Text)[0]
            except:
                 print("Never Mind: COULD NOT FIND {} IN THE PAGE WITH THE TEXT".format(End_Text))
            Chapter = Chapter + Text
                
        else:
            Chapter = Chapter + Text   
    return(Chapter)

In [190]:
Chapter_Texts = []
for i in range(len(PositiveChaptersDF)):
    try:
        if PositiveChaptersDF.iloc[i+1]['PDF_Found_Last'] == PositiveChaptersDF.iloc[i]['PDF_Found_Last']:
            End_Page = PositiveChaptersDF.iloc[i+1]['Page_Found_Last']
            End_Text = PositiveChaptersDF.iloc[i+1]['Chapter_Keys']
        else:
            # For cases when the chapter in the last chapter in the outline for that 
            End_Page = 99999
            End_Text = "Vibudh loves chocolates." # Random Text which is not there in the PDF 
        
    except:
        # For case when the chapter in the last chapter in the DataFrame  
        End_Page = 99999
        End_Text = "Vibudh loves chocolates." # Random Text which is not there in the PDF 
        
    Chapter_Text = read_chapter_text(PositiveChaptersDF.iloc[i]['PDF_locations'], 
                      PositiveChaptersDF.iloc[i]['Chapter_Keys'],
                      PositiveChaptersDF.iloc[i]['Page_Found_Last'],
                      End_Page, End_Text)
                      
    Chapter_Texts.extend([Chapter_Text])
        
len(Chapter_Texts)
len(PositiveChaptersDF)

Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf
Never Mind: COULD NOT FIND Cover IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q4.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q4.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q4.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q4.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q4.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F

Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q6.pdf
Never Mind: COULD NOT FIND Figure 11.1-33: Moose Habitat Suitability Brewster Section – North IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q6.pdf
Never Mind: COULD NOT FIND Figure 11.1-34: Moose Habitat Suitability Brewster Section – South IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q6.pdf
Never Mind: COULD NOT FIND Figure 11.1-35: Moose Habitat Suitability Didsbury Compressor Station Unit Addition IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q6.pdf
Never Mind: COULD NOT FIND Figure 11.1-36: Moose Habitat Suitability Beiseker Compressor Unit Addition IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q6.pdf
Never Mind: COULD NOT FIND Figure 11.1-37: Canada Lynx Habitat Suitability Valhalla Section IN THE PAGE WITH THE TEXT
Reading PDF

Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q6.pdf
Never Mind: COULD NOT FIND Figure 11.1-75: Regional Wildlife Sensitivities Karr Section IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q6.pdf
Never Mind: COULD NOT FIND Figure 11.1-76: Regional Wildlife Sensitivities Karr and Deep Valley Sections IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q6.pdf
Never Mind: COULD NOT FIND Figure 11.1-77: Regional Wildlife Sensitivities Deep Valley Section IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q6.pdf
Never Mind: COULD NOT FIND Figure 11.1-78: Regional Wildlife Sensitivities Deep Valley and McLeod River Connection Sections IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q6.pdf
Never Mind: COULD NOT FIND Figure 11.1-79: Regional Wildlife Sensitivities Robb Section IN THE PAGE WITH THE TEXT
Readi

Never Mind: COULD NOT FIND Noise Impact Assessment Technical Data Report IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R2.pdf
Never Mind: COULD NOT FIND Appendix I: Preliminary Caribou Habitat Restoration and Offset Measurement Plan (CHROMP) IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R2.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R2.pdf
Never Mind: COULD NOT FIND Appendix J: Preliminary Decommissioning and Abandonment Plan IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R2.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R2.pdf
Never Mind: COULD NOT FIND Appendix K: Traditional Knowledge Report IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R2.pdf
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R2.pdf


176

176

# Rough Work

In [189]:
Echap = 0
for i in Chapter_Texts: 
    if i == "":
        Echap = Echap + 1
        print("Empty Chap")
        
Echap

0

In [118]:
a = ""
b = "vob"
a = a +b 
a

Start_Text = "SA"
Text = "Vibudh SAingh"
Text = Text.split(Start_Text)[1]
Text + b 
PositiveChaptersDF.iloc[8000]['Chapter_Keys']

'vob'

'inghvob'

'4.0 ASSESSMENT AREAS'

In [125]:
for i in range(8000, len(PositiveChaptersDF)):
    try:
        if PositiveChaptersDF.iloc[i+1]['PDF_Found_Last'] == PositiveChaptersDF.iloc[i]['PDF_Found_Last']:
            End_Page = PositiveChaptersDF.iloc[i+1]['Page_Found_Last']
            End_Text = PositiveChaptersDF.iloc[i+1]['Chapter_Keys']
        else:
            # For cases when the chapter in the last chapter in the outline for that 
            End_Page = 99999
            End_Text = "Vibudh Rocks" # Random Text which is not there in the PDF 
        
    except:
        # For case when the chapter in the last chapter in the DataFrame  
        End_Page = 99999
        End_Text = "Vibudh Rocks" # Random Text which is not there in the PDF 
        
    Chapter_Text = read_chapter_text(PositiveChaptersDF.iloc[i]['PDF_locations'], 
                      PositiveChaptersDF.iloc[i]['Chapter_Keys'],
                      PositiveChaptersDF.iloc[i]['Page_Found_Last'],
                      End_Page, End_Text)
                      
    Chapter_Texts.extend([Chapter_Text])
        
len(Chapter_Texts)
len(PositiveChaptersDF)

Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2W4.pdf
*****************************************
*****************************************
*****************************************
The Text Extracted for page no 460 is FINAL
 REPORT
 rwdi.com
 This 
document is intended for the sole use of the party to whom it is addressed and may contain information that is privileged and
/or confidential. If you 
have received this in error, please notify us immediately. ® RWDI name and logo are registered trademarks 
in Canada and the United States of America
  NORTH CORRIDOR 
EXPANSION PROJECT
  NOISE IMPACT ASSESSM
ENT
 RWDI #
1803694
 April 1, 2019
   SUBMITTED TO
 Barbara Taylor
 TransCanada
 450 1st St SW
 Calgary, Alberta
 T2P 5H1
 barbara_taylor
@transcanada.com
    SUBMITTED BY
 Candace Bell, M.Sc.
 Project Manager
 Candace.Bell
@rwdi.com
     RWDI
 Suite 1000, 736
-8th Avenue S.W.
 Calgary, Alberta, Canada
 T2P 1H4
 T: 
403.232.6771
 F: 
519.823.1316
    
Never Mind
******

The Text Extracted for page no 466 is NOISE IMPACT ASSESSM
ENT
 NORTH CORRIDOR EXPAN
SION PROJECT
 RWDI#
1803694
 April 1, 2019
  rwdi.com
   
Page 
4 
     ﬁdaytimeﬂ adjustment is used for the hours between 07:00 and 22:00. The daytime PSL would include a +10 dBA 
adjustment to the nighttime PSL as defined Directive 038.
 Where no noise sensitive receptors have been identified, the 4
0 dBA nighttime L
EQ limit, 
at the 1.5 km criteria 
boundary defined in 
AER 
Directive 038 will be used.
 As stated in AER 
Directive 038
, 
the natural ambient sound level (ASL) is generally defined as 5 dBA less than the 
BSL
. Adjustments for the presence of 
non
-p
roject related ambient sound levels are defined in 
AER Directive 038
. These
 adjustments are applied as such situations are identified in the study 
based on
 the baseline noise 
measurements
 and expected operating conditions
.  With the cumulative approach that is required, compliance of predicted 
project sound levels with the PSL cr

The Text Extracted for page no 469 is NOISE IMPACT ASSESSM
ENT
 NORTH CORRIDOR EXPAN
SION PROJECT
 RWDI#
1803694
 April 1, 2019
  rwdi.com
   
Page 
7 
     4 FACILITY SOUND SOURCES
 4.1
 Adjacent
 Facilities
 To identify facilities that may contribute to the acoustic environment
 near 
the 
Hidden Lake
 North 
Unit 
Addition
, publicly available databases were reviewed to determine the locations of potential third
-party 
energy
-related 
contributors. The following databases were consulted to determine the locations of facilities in the vicinity of the 
Hidden Lake North 
Unit Addition
: 
 Alberta Energy Regulator ST37 
Œ 
Alberta well listing (AER
 2018
a)  Alberta Energy Regulator ST102 
Œ 
Alberta facility listing (AER
 2018
b)
 The f
acilities shown in Table 2
 were further 
verified 
by a field survey and operation was confirmed by follow
-up 
with CNRL.  The 
Hidden Lake North Unit Addition 
and existing facilities are shown in 
Figure 
1. 
Table 
2: Adjacent
 Facilities Consid

The Text Extracted for page no 474 is NOISE IMPACT ASSESSM
ENT
 NORTH CORRIDOR EXPAN
SION PROJECT
 RWDI#
1803694
 April 1, 2019
  rwdi.com
            
Page 
12      Item
 Source 
Type
1 QTY
 Levels at Octave Band Center Frequencies (dB)
 Overall Sound Power
 Reference 
Source
 31.5
 63
 125
 250
 500
 1,000
 2,000
 4,000
 8,000
 (dBA)
 (dB)
 Bld Louver Set 3
 P 
1 
79.7
 79.6
 83.5
 83.6
 87.3
 87.0
 93.3
 90.9
 85.8
 97.4
 97.3
 (3)(4)(5)
 Lube Oil Cooler
 P 
1 
104.5
 111.5
 108.5
 101.5
 96.5
 93.5
 89.5
 85.5
 80.5
 100.0
 114.2
 (3)(4)(5)
 Bld Roof
 A 
1 
99.6
 98.6
 100.4
 96.6
 91.2
 88.0
 92.2
 85.9
 78.8
 96.9
 105.5
 (3)(4)(5)
 Bld Wall
 V 
2 
96.6
 95.6
 97.4
 93.6
 88.2
 85.0
 89.2
 82.9
 75.8
 93.9
 102.6
 (3)(4)(5)
 Bld Wall
 V 
2 
97.6
 96.6
 98.4
 94.6
 89.2
 86.0
 90.2
 83.9
 76.8
 94.9
 103.5
 (3)(4)(5)
 Hidden Lake Compressor Station 
- 
Control Building (Boiler Running Only, Genset Off)
 Boiler Exhaust
 P 
1 
92.1
 91.1
 88.1
 86.1
 74.1
 67.1
 59.1
 56.1
 42.1
 79

The Text Extracted for page no 481 is 































*****************************************
*****************************************
*****************************************
The Text Extracted for page no 482 is NOISE IMPACT ASSESSM
ENT
 NORTH CORRIDOR EXPAN
SION PROJECT
 RWDI#
1803694
 April 1, 2019
  rwdi.com
 Page 
20   5.4 Low Frequency Noise
 The C
-weighted sound level (dBC) results have been reviewed to determine if there is a potential for LFN at the 
boundary receptor. The differences between the 
Hidden Lake North Unit Addition
-only C
-weighted and 
A-
weighted values at the chosen boundary receptor is shown in 
Table 
8. 
Table 
8: Low Frequency Noise Assessment 
 Receptor ID
 C-Weighted 
Sound Level
 (dBC)
 A-Weighted 
Sound Level
 (dBA)
 dBC
-dBA
 Potential LFN 
Condition?
 (Yes/No)
 BR1
 60.6
 36.1
 24.5
 Yes
  The 
calculation for LFN shows l
evels above the AER threshold.
 However, 
because
 the 
facility is in a remote 
location with no dwelling


...

*****************************************
*****************************************
*****************************************
The Text Extracted for page no 206 is  Appendix 12. Preliminary 
Decommissioning or Abandonment
 Assessment
  A12
-6 AX0917181256CGY
 abandonment will be in accordance with the most current body of scientific knowledge and NGTL™s 
established and proven mitigation measures and construct
ion practices available at the time of 
decommissioning or abandonment. 
 Land disturbance activities related to the removal of pipeline or associated facilities would be reclaimed to 
the equivalent land use at that time. The methods for evaluating the succe
ss of reclamation would be 

based on the principle that the success of land reclamation is measured against the adjacent site 
conditions, while taking into consideration the status of reclamation at the time of the assessment. 
 The reclamation objectives or
 principles to be applied to abandonment of the Project will also

The Text Extracted for page no 204 is  Appendix 12. Preliminary 
Decommissioning or Abandonment
 Assessment
  A12
-4 AX0917181256CGY
 Table 
A12
-1. 
Potential 
Element Interaction
s with the North Corridor Expansion Project
 Biophysical and 
Socio
-Economic 
Element
 Interaction (Yes / No)
 Description of Potential 
Interactions During 
Decommissioning and Abandonment Activities
 Pipeline
 (North 
Star Section 2,
 Red Earth 
Section 3,
 and
 Bear Canyon 
North 
Extension
) Facilities 
(including 
Hidden Lake 
North 
Unit 
Addition)
 Heritage Resources
 No No No heritage resources are anticipated on the previously 
disturbed 
PCF
. Traditional Land and 
Resource Use
 Yes
 Yes
 Decommissioning and abandonment activities could alter or 
temporarily disrupt use of trails and travelways,
 and subsistence activities.
 Social and Cultural 
Well
-being
 Yes
 Yes
 Decommissioning and abandonment activities could 
temporarily affect the local population due to the influx of 
temporary workforce

The Text Extracted for page no 206 is  Appendix 12. Preliminary 
Decommissioning or Abandonment
 Assessment
  A12
-6 AX0917181256CGY
 abandonment will be in accordance with the most current body of scientific knowledge and NGTL™s 
established and proven mitigation measures and construct
ion practices available at the time of 
decommissioning or abandonment. 
 Land disturbance activities related to the removal of pipeline or associated facilities would be reclaimed to 
the equivalent land use at that time. The methods for evaluating the succe
ss of reclamation would be 

based on the principle that the success of land reclamation is measured against the adjacent site 
conditions, while taking into consideration the status of reclamation at the time of the assessment. 
 The reclamation objectives or
 principles to be applied to abandonment of the Project will also be in 
accordance with legislative and regulatory requirements in place at that time. The primary goal of 
reclamation follow

210

8122

In [95]:
PCDF = PositiveChaptersDF[8100:]
PCDF
len(PCDF)

,PDF_names,PDF_locations,Chunk_Nos,Chapter_Numbers,Chapter_Keys,Chapter_levels,PDF_Found_Last,Page_Found_Last
12742,A6T2W4.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,NOISE IMPACT ASSESSMENT,3,A6T2W4.pdf,460
12752,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Appendix 6 Aquatics Technical Data Report,2,A6T2W5.pdf,0
12753,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Aquatics Technical Data Report for the Propose...,3,A6T2W5.pdf,1
12763,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Appendix 7 Wetland Technical Data Report,2,A6T2W5.pdf,59
12764,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Wetland Technical Data Report for the Proposed...,3,A6T2W5.pdf,60
12772,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Appendix 8 Vegetation Technical Data Report,2,A6T2W5.pdf,82
12773,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Vegetation Technical Data Report for the Propo...,3,A6T2W5.pdf,83
12783,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Appendix 9 Wildlife Technical Data Report,2,A6T2W5.pdf,155
12784,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Wildlife Technical Data Report for the Propose...,3,A6T2W5.pdf,156
12795,A6T2W6.pdf,F:/Environmental Baseline Data/Version 3/Data/...,286,0,Appendix 10 Caribou Habitat Restoration and Of...,2,A6T2W6.pdf,0


22

In [ ]:
object = p2.PdfFileReader(PDF_location)
        
    # get number of pages
    NumPages = object.getNumPages()
    if NumPages < End_Page:
        End_Page = NumPages - 1
    
    Chapter = ""    
    for i in range(Start_Page, End_Page+1):
        PageObj = object.getPage(i)
        
        try:
            if i == Start_Page:
                Text = PageObj.extractText()
                try:
                    Text = Text.split(Start_Text)[1]
                except:
                    print("Never Mind")
                Chapter.append(Start_Text + Text)

In [86]:
PDF_location = PositiveChaptersDF.iloc[1]['PDF_locations']
object = p2.PdfFileReader(PDF_location)
# get number of pages
NumPages = object.getNumPages()

NumPages

PageObj = object.getPage(27)
Text = PageObj.extractText()
Text

107

' ENVIRONMENTAL \nIMPACT \nASSESSMENTAND \nMITIGATION \nPLAN\n Page 13-4\n   EnCana Ekwan Pipeline Inc. \n13.2 Project Description \n13.2.1 Scope of Project \nThis Project will involve the construction and oper\nation of approximately \n83 km of 61 cm \n(51 miles of 24 inch) pipeline. \nNo upstream modifications to \nthe EnCana Sierra Gas Plant \nor downstream modifications to the NGTL \nNorthwest Mainline pipeline system will be \nrequired to accommodate the new gas flow \nthrough the proposed Pipeline. Therefore, based on the ‚principal project/a\nccessory test™ provided by the \nCanadian Environmental \nAssessment Act \nTraining Compendium (Canadian Envi\nronmental Assessment Agency \n1994), the ‚scope of the project™ \nfrom the perspective of the \nCanadian Environmental Assessment Act \ncan be defined as the physical works \nassociated with the construction and \noperation of the Pipeline only.  Project details are provided in Volume 1 of\n the Application document. The informatio

In [ ]:
Chapter_Texts = []
for i in range(8100, len(PositiveChaptersDF)):
    try:
        if PositiveChaptersDF.iloc[i+1]['PDF_Found_Last'] == PositiveChaptersDF.iloc[i]['PDF_Found_Last']:
            End_Page = PositiveChaptersDF.iloc[i+1]['Page_Found_Last']
            End_Text = PositiveChaptersDF.iloc[i+1]['Chapter_Keys']
        else:
            # For cases when the chapter in the last chapter in the outline for that 
            End_Page = 99999
            End_Text = "Vibudh Rocks" # Random Text which is not there in the PDF 
        
    except:
        # For case when the chapter in the last chapter in the DataFrame  
        End_Page = 99999
        End_Text = "Vibudh Rocks" # Random Text which is not there in the PDF 
        
    Chapter_Text = read_chapter_text(PositiveChaptersDF.iloc[i]['PDF_locations'], 
                      PositiveChaptersDF.iloc[i]['Chapter_Keys'],
                      PositiveChaptersDF.iloc[i]['Page_Found_Last'],
                      End_Page, End_Text)
                      
    Chapter_Texts.extend([Chapter_Text])
        
len(Chapter_Texts)
len(PositiveChaptersDF)

In [ ]:
['Username', ' How are you today?']

In [65]:
s1 = "Vibudh Rocks. He is awesome"
s1.split('%%%%')[0]
s1.split('He')[1]

'Vibudh Rocks. He is awesome'

' is awesome'

In [70]:
for i in range(5,5):
    print("Viobud")

In [ ]:
# Variables for Initialization of the data frame for Index 4 - Chapters.csv
file_names = []
found_file_names = []
chapters = []
levels = []
pages = []
all_df_chapters = pd.DataFrame({'file_names': file_names, 'found_file_names': found_file_names, 'chapters': chapters, 'pages' : pages, 'levels' : levels})

for current_chunk in range(10,12):#chunk+1):
    current_df_outlines = df_outlines[df_outlines.chunk_nos == current_chunk]
    
    chunk_has_outline = 0
    chunk_noOutline_hasTOC = 0
    
    file_name_containing_index = '0'
    
    p_file_names = []
    p_found_file_names = []
    p_chapters = []
    p_levels = []
    p_pages = []
    level = 0
    # Going through all the PDFs - Chunk by Chunk
    for i in range(len(current_df_outlines)):
        # First file in the chunk has Outline 
        if current_df_outlines.iloc[i]['outline_present'] == 1:
            chunk_has_outline = 1
            print("stage 1")
            pdfread = p2.PdfFileReader(current_df_outlines.iloc[i]['file_paths'])
            s = pdfread.outlines
            
            nested_is_instance_check(s, 
                                     level, 
                                     current_df_outlines.iloc[i]['file_names'], 
                                     p_file_names, p_found_file_names, 
                                     p_chapters, p_levels, p_pages)
            file_name_containing_index = current_df_outlines.iloc[i]['file_names']
            continue
            
        if count_occorances(p_pages, -999) > 1 and chunk_has_outline == 1:
            print("stage 2: file is {}".format(current_df_outlines.iloc[i]['file_names']))
            start_flag = 0
            page_no = -999
            #print(p_pages)
            for j in range(len(p_chapters)):
                if p_pages[j] >= 0 and p_levels[j] < 5: # probably could have also used p_pages[j] >= - 998
                    print("start flag is on")
                    start_flag = 1
                    continue
                
                if start_flag != 1:
                    continue
                    
                if start_flag == 1 and p_pages[j] <= 0 and p_levels[j] < 5:
                    print("stage 3: finding page no for {} with page{}".format(p_chapters[j], p_pages[j]))
                    page_no, found_file_name = find_page_no(current_df_outlines.iloc[i]['file_paths'], 
                                                            p_chapters[j])
                    p_pages[j] = page_no
                    p_found_file_names = current_df_outlines.iloc[i]['file_names']
                    
                if page_no == -999 and p_levels[j] < 5:
                    print("Not found")
                    break # if the first keyword is not found then dont look any further
                else:
                    print("New Found ")
            
    df_chapters = pd.DataFrame({'file_names': p_file_names, 'found_file_names': p_found_file_names, 'chapters': p_chapters, 'pages' : p_pages, 'levels' : p_levels})
    all_df_chapters = pd.concat([all_df_chapters, df_chapters])
        
    
    